# Homework 3

Use this resource as a reference: https://cryptobook.nakov.com/digital-signatures/ecdsa-sign-verify-messages

The following may also be helpful:

https://www.rareskills.io/post/finite-fields

https://www.rareskills.io/post/elliptic-curve-addition

https://www.rareskills.io/post/elliptic-curves-finite-fields

Implement ECDSA from scratch. You want to use the secp256k1 curve. When starting off, use the Elliptic curve multiplication library used in the blog post linked here: https://www.rareskills.io/post/generate-ethereum-address-from-private-key-python

1) pick a private key

2) generate the public key using that private key (not the eth address, the public key)

3) pick message m and hash it to produce h (h can be though of as a 256 bit number)

4) sign m using your private key and a randomly chosen nonce k. produce (r, s, h, PubKey)

5) verify (r, s, h, PubKey) is valid

You may use a library for point multiplication, but everything else you must do from scratch.

Pay close attention to the distinction between the curve order and the prime number $p$ we compute the modulus of $y^2=x^3+b \pmod p$.

In [ ]:
!python -m pip install ecpy
!python -m pip install pycryptodome

In [2]:
from Crypto.Hash import keccak

# 1. Pick a private key and a message m, and a nonce
private_key = 0xac0974bec39a17e36ba4a6b4d238ff944bacb478cbed5efcae784d7bf4f2ff80
m = "New year resolution: First bar muscle up"
nonce = 98360938612402479070358389144388875689143220897586497735012657195388739476407 # ecrand.rnd(pri_key.curve.order)
print("Nonce: ", nonce)

Nonce:  98360938612402479070358389144388875689143220897586497735012657195388739476407


In [3]:
from ecpy.curves import Curve
from ecpy.keys import ECPublicKey, ECPrivateKey
from ecpy.ecdsa import ECDSA
# from ecpy import ecrand

############################################
# First use a library to go through the steps
############################################

# 2. Generate the public key from ellipic curve
cv = Curve.get_curve('secp256k1')
pri_key = ECPrivateKey(private_key, cv)
pub_key = pri_key.get_public_key()
# Print the public key
print("Private key: ", pri_key)
print("Public key: ", pub_key)
# (irrelevant to the exercise but just to show the ethereum address)
concat_x_y = pub_key.W.x.to_bytes(32, byteorder='big') + pub_key.W.y.to_bytes(32, byteorder='big')
hash = keccak.new(digest_bits=256)
hash.update(concat_x_y)
address = "0x" + hash.hexdigest()[-40:]
# 0xf39Fd6e51aad88F6F4ce6aB8827279cffFb92266
print("Address: ", address) 

# hash the message m to produce a digest h
print("Message: ", m)
h = keccak.new(digest_bits=256)
h.update(m.encode())
print("Digest: ", h.digest().hex())

# 3. Sign the digest
signer = ECDSA()
signature = signer._do_sign(h.digest(), pri_key, nonce)
print("Signature: ", signature.hex())

# 4. Verify the signature
print("Verification: ", signer.verify(h.digest(), signature, pub_key))

Private key:  ECPrivateKey:
  d: ac0974bec39a17e36ba4a6b4d238ff944bacb478cbed5efcae784d7bf4f2ff80
Public key:  ECPublicKey:
  x: 8318535b54105d4a7aae60c08fc45f9687181b4fdfc625bd1a753fa7397fed75
  y: 3547f11ca8696646f2f3acb08e31016afac23e630c5d11f59f61fef57b0d2aa5
Address:  0xf39fd6e51aad88f6f4ce6ab8827279cfffb92266
Message:  New year resolution: First bar muscle up
Digest:  9680b95ec7c25b5720330d7bc8a413dcf46f0fd43aa16ad60673bf9ec191cbf7
Signature:  3045022100afa434a9b68b9a0d29a2d1e6796c993a4a1326b39fe16a5558e3b108db383039022012e87338e97e1ea83747170c5712f3fad025dd9c21a8f0d88aed2cdd3165fa9d
Verification:  True


In [4]:
############################################
# Next build everything from scratch
############################################

from ecpy.curves import Curve

# Basics of elliptic curve secp256k1
secp256k1_n = 115792089237316195423570985008687907852837564279074904382605163141518161494337
Gx = 55066263022277343669578718895168534326250603453777594175500187360389116729240
Gy = 32670510020758816978083085130507043184471273380659243275938904335757337482424
p = 2**256 - 2**32 - 977 # FFFFFFFF FFFFFFFF FFFFFFFF FFFFFFFF FFFFFFFF FFFFFFFF FFFFFFFE FFFFFC2F

cv  = Curve.get_curve('secp256k1')

assert cv.order == secp256k1_n
assert cv.generator.x == Gx
assert cv.generator.y == Gy
assert cv.field == p

# 1. Pick a private key
sk = private_key
assert 0 < sk < secp256k1_n 

# 2. Generate the public key from ellipic curve
pk = sk * cv.generator
print("Public key: ", pk)

#############
# ECDSA sign
#############
# get digest
msg_hash = keccak.new(digest_bits=256)
msg_hash.update(m.encode())
print("Signer:")
print("Digest: ", msg_hash.digest().hex())
assert msg_hash.digest() == h.digest()
# get nonce
k = nonce
assert 0 < k < secp256k1_n

# random point from nonce
R = k * cv.generator
r = R.x
print("R: ", R)
print("r: ", r)
assert 0 < r < secp256k1_n

# s = k^-1 (msg_hash + r * sk) mod secp256k1_n
k_inv = pow(k, -1, secp256k1_n)
print("k_inv: ", k_inv)
assert 0 < k_inv < secp256k1_n
assert k * k_inv % secp256k1_n == 1

s = pow(k, -1, secp256k1_n) * (int(msg_hash.hexdigest(), 16) + r * sk) % secp256k1_n
print("s: ", s)

# The calculated signature {r, s} is a pair of integers, each in the range [1...n-1]. 
# It encodes the random point R = k * G, along with a proof s, 
# confirming that the signer knows the message h and the private key privKey.

##############
# ECDSA verify
##############
# hash the message
msg_hash_verify = keccak.new(digest_bits=256)
msg_hash_verify.update(m.encode())
print("Verifier:")
print("Digest: ", msg_hash_verify.digest().hex())
assert msg_hash_verify.digest() == msg_hash.digest()

# calculate the mod inverse of the signature proof s1 = s^-1 mod n
s_inv = pow(s, -1, secp256k1_n)
print("s_inv: ", s_inv)
assert 0 < s_inv < secp256k1_n
assert s * s_inv % secp256k1_n == 1

# revover the random point used dring the signing R' = msg_hash * s^-1 * G + r * s^-1 * pk
R_prime = int(msg_hash.hexdigest(), 16) * s_inv * cv.generator + r * s_inv * pk
print("R_prime: ", R_prime)
r_prime = R_prime.x

# The verifier then checks that the calculated point R' is equal to the original point R.
assert r_prime == r

Public key:  (0x8318535b54105d4a7aae60c08fc45f9687181b4fdfc625bd1a753fa7397fed75 , 0x3547f11ca8696646f2f3acb08e31016afac23e630c5d11f59f61fef57b0d2aa5)
Signer:
Digest:  9680b95ec7c25b5720330d7bc8a413dcf46f0fd43aa16ad60673bf9ec191cbf7
R:  (0xafa434a9b68b9a0d29a2d1e6796c993a4a1326b39fe16a5558e3b108db383039 , 0xc035b099fdd4adf88dc3eea4d33c1c73505dfe9017f34d3df4e5f82bf26acbde)
r:  79444874886960558771284831188131706133134540605303047920221900133408563540025
k_inv:  108619977168711122927283479210662165891967813454229439905135531601682982445623
s:  8552335028703921046833810355767488604052056988320138045475493806081013316253
Verifier:
Digest:  9680b95ec7c25b5720330d7bc8a413dcf46f0fd43aa16ad60673bf9ec191cbf7
s_inv:  103152377218759835340058802868463102436846175229107224233394657806873677105197
R_prime:  (0xafa434a9b68b9a0d29a2d1e6796c993a4a1326b39fe16a5558e3b108db383039 , 0xc035b099fdd4adf88dc3eea4d33c1c73505dfe9017f34d3df4e5f82bf26acbde)


In [8]:
# what if the signer wants to trick the verifier by using the mirror point of R?!
k_mirror = -k % secp256k1_n
print("k_mirror: ", k_mirror)
R_mirror = k_mirror * cv.generator
r_mirror = R_mirror.x
print("R_mirror: ", R_mirror)
print("r_mirror: ", r_mirror)
assert 0 < r_mirror < secp256k1_n
assert r_mirror == r

# Signer produces another signature {r_mirror, s_mirror} using the mirror point R_mirror
s_mirror = pow(k_mirror, -1, secp256k1_n) * (int(msg_hash.hexdigest(), 16) + r_mirror * sk) % secp256k1_n
print("s_mirror: ", s_mirror)

# Verifier checks the signature {r_mirror, s_mirror} using the mirror point R_mirror
s_mirror_inv = pow(s_mirror, -1, secp256k1_n)
R_mirror_prime = int(msg_hash.hexdigest(), 16) * s_mirror_inv * cv.generator + r_mirror * s_mirror_inv * pk
r_mirror_prime = R_mirror_prime.x
assert r_mirror_prime == r

# However, the y values ain't the same!
assert R_mirror_prime.y != R.y


k_mirror:  17431150624913716353212595864299032163694343381488406647592505946129422017930
R_mirror:  (0xafa434a9b68b9a0d29a2d1e6796c993a4a1326b39fe16a5558e3b108db383039 , 0x3fca4f66022b5207723c115b2cc3e38cafa2016fe80cb2c20b1a07d30d953051)
r_mirror:  79444874886960558771284831188131706133134540605303047920221900133408563540025
s_mirror:  107239754208612274376737174652920419248785507290754766337129669335437148178084


In [11]:
################
# Point recovery
################
from ecpy.curves import Point

# recover R from r (which is Rx)
y_recovered_1 = cv.y_recover(r, 0)
y_recovered_2 = cv.y_recover(r, 1)

print("y_recovered_1: ", y_recovered_1)
print("y_recovered_2: ", y_recovered_2)

# recovered curve points
R_recovered_1 = Point(r, y_recovered_1, cv)
R_recovered_2 = Point(r, y_recovered_2, cv)

print("R_recovered_1: ", R_recovered_1)
print("R_recovered_2: ", R_recovered_2)

assert R_recovered_1.is_on_curve
assert R_recovered_2.is_on_curve

y_recovered_1:  86938928681380777244582586478024060883314179233276832289830260104917192199134
y_recovered_2:  28853160555935418178988398530663846969955805432363731749627323902991642472529
R_recovered_1:  (0xafa434a9b68b9a0d29a2d1e6796c993a4a1326b39fe16a5558e3b108db383039 , 0xc035b099fdd4adf88dc3eea4d33c1c73505dfe9017f34d3df4e5f82bf26acbde)
R_recovered_2:  (0xafa434a9b68b9a0d29a2d1e6796c993a4a1326b39fe16a5558e3b108db383039 , 0x3fca4f66022b5207723c115b2cc3e38cafa2016fe80cb2c20b1a07d30d953051)


In [23]:
## Find the nonce difference when a signer signs the exact same message twice with two different nonces
# a signer picked a nonce k_1 and produces a signature {r_1, s_1}
k_1 = 98360938612402479070358389144388875689143220897586497735012657195388739476407
r_1 = (k_1 * cv.generator).x
s_1 = pow(k_1, -1, secp256k1_n) * (int(msg_hash.hexdigest(), 16) + r_1 * sk) % secp256k1_n
print("r_1", r_1)
print("s_1", s_1)

# a verifier checks the signature {r_1, s_1}
s_1_inv = pow(s_1, -1, secp256k1_n)
R_1_prime = int(msg_hash.hexdigest(), 16) * s_1_inv * cv.generator + r_1 * s_1_inv * pk
assert R_1_prime.x == r_1

# later the signer picks another nonce k_2 and produces a signature {r_2, s_2} 
increment = 256
# nonce k2 is k1 + increment
k_2 = k_1 + increment
assert 0 < k_2 < secp256k1_n
r_2 = (k_2 * cv.generator).x
s_2 = pow(k_2, -1, secp256k1_n) * (int(msg_hash.hexdigest(), 16) + r_2 * sk) % secp256k1_n
print("r_2", r_2)
print("s_2", s_2)

# the verifier checks the signature {r_2, s_2}
s_2_inv = pow(s_2, -1, secp256k1_n)
R_2_prime = int(msg_hash.hexdigest(), 16) * s_2_inv * cv.generator + r_2 * s_2_inv * pk
assert R_2_prime.x == r_2

# what's interesting is that by recovering R1 and R2 with r_1 and r_2, we could find the nonce difference
R_1_recovered_0 = Point(r_1, cv.y_recover(r_1, 0), cv)
R_1_recovered_1 = Point(r_1, cv.y_recover(r_1, 1), cv)
R_2_recovered_0 = Point(r_2, cv.y_recover(r_2, 0), cv)
R_2_recovered_1 = Point(r_2, cv.y_recover(r_2, 1), cv)

print("R_1_recovered_0: ", R_1_recovered_0)
print("R_1_recovered_1: ", R_1_recovered_1)
print("R_2_recovered_0: ", R_2_recovered_0)
print("R_2_recovered_1: ", R_2_recovered_1)

nonce_diff_0_0 = 0
nonce_diff_0_1 = 0
nonce_diff_1_0 = 0
nonce_diff_1_1 = 0

# nonce difference
for i in range(1, secp256k1_n):
    diff = cv.generator * i
    
    if nonce_diff_0_0 == 0 and R_1_recovered_0.add(diff) == R_2_recovered_0:
        nonce_diff_0_0 = i
        break
    if nonce_diff_0_1 == 0 and R_1_recovered_0.add(diff) == R_2_recovered_1:
        nonce_diff_0_1 = i
        break
    if nonce_diff_1_0 == 0 and R_1_recovered_1.add(diff) == R_2_recovered_0:
        nonce_diff_1_0 = i
        break
    if nonce_diff_1_1 == 0 and R_1_recovered_1.add(diff) == R_2_recovered_1:
        nonce_diff_1_1 = i
        break

print("Nonce difference: ", nonce_diff_0_0, nonce_diff_0_1, nonce_diff_1_0, nonce_diff_1_1)

# Find the increment
assert increment == max(nonce_diff_0_0, nonce_diff_0_1, nonce_diff_1_0, nonce_diff_1_1)
assert R_1_recovered_0.add(cv.generator * increment) == R_2_recovered_1

r_1 79444874886960558771284831188131706133134540605303047920221900133408563540025
s_1 8552335028703921046833810355767488604052056988320138045475493806081013316253
r_2 22191701995451972799410659152280904163162905382785057495695234338083165812608
s_2 15963876287145823181850471826568451079066389864545327526043744142493001256140
R_1_recovered_0:  (0xafa434a9b68b9a0d29a2d1e6796c993a4a1326b39fe16a5558e3b108db383039 , 0xc035b099fdd4adf88dc3eea4d33c1c73505dfe9017f34d3df4e5f82bf26acbde)
R_1_recovered_1:  (0xafa434a9b68b9a0d29a2d1e6796c993a4a1326b39fe16a5558e3b108db383039 , 0x3fca4f66022b5207723c115b2cc3e38cafa2016fe80cb2c20b1a07d30d953051)
R_2_recovered_0:  (0x31100ee75b86f1239e224dac3bfc8d553be85cd36e9978c514af1dc5654db780 , 0xa3f86408b90bbf91ccf132e329f573bc555414573b75801c8920f39c551ec9b8)
R_2_recovered_1:  (0x31100ee75b86f1239e224dac3bfc8d553be85cd36e9978c514af1dc5654db780 , 0x5c079bf746f4406e330ecd1cd60a8c43aaabeba8c48a7fe376df0c62aae13277)
Nonce difference:  0 256 0 0
